In [1]:
!pip install --upgrade --user google-cloud-aiplatform google-cloud-storage
!pip install langchain_community
!pip install google-cloud-aiplatform
!pip install google-cloud-storage
!pip install langchain_community
!pip install langchain
!pip install pypdf
!pip install google-cloud-firestore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 18.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of google-api-core[grpc] to determine which version is compatible with other requirements. This could take a while.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.1 MB/s eta 0:00:00


In [1]:
import IPython
from IPython.display import Markdown, display
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
# get project ID
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"

# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")

In [4]:
!gcloud services enable \
  aiplatform.googleapis.com \
  storage.googleapis.com \
  run.googleapis.com \
  compute.googleapis.com



Operation "operations/acf.p2-619606649119-aa15bfa9-3b61-4fee-921c-645dadf58fa6" finished successfully.


In [8]:
!gcloud config set project qwiklabs-gcp-00-c908ca7e2977
!gcloud firestore databases create \
  --project=qwiklabs-gcp-00-c908ca7e2977 \
  --location=nam5 \
  --type=firestore-native


Updated property [core/project].
metadata:
  '@type': type.googleapis.com/google.firestore.admin.v1.CreateDatabaseMetadata
name: projects/qwiklabs-gcp-00-c908ca7e2977/databases/(default)/operations/WA5mIlQVzpKtRecoWbwUlBAqNW1hbgQiBRBbmJ7wEAaxhabyCAsKJxo
response:
  '@type': type.googleapis.com/google.firestore.admin.v1.Database
  appEngineIntegrationMode: DISABLED
  concurrencyMode: PESSIMISTIC
  createTime: '2024-04-18T17:08:02.191238Z'
  deleteProtectionState: DELETE_PROTECTION_DISABLED
  earliestVersionTime: '2024-04-18T17:08:02.191238Z'
  etag: IPmDppygzIUDMIbXpJygzIUD
  keyPrefix: s
  locationId: nam5
  name: projects/qwiklabs-gcp-00-c908ca7e2977/databases/(default)
  pointInTimeRecoveryEnablement: POINT_IN_TIME_RECOVERY_DISABLED
  type: FIRESTORE_NATIVE
  uid: 9414bc59-28e7-45ad-92ce-155422660e58
  updateTime: '2024-04-18T17:08:02.191238Z'
  versionRetentionPeriod: 3600s


In [70]:
def create_data_packet(file_name, file_type, page_number, file_content):
    """Creating a simple dictionary to store all information (content and metadata)
    extracted from the document"""
    data_packet = {}
    data_packet["file_name"] = file_name
    data_packet["file_type"] = file_type
    data_packet["page_number"] = page_number
    data_packet["content"] = file_content
    return data_packet

In [63]:
!mkdir test
!gsutil -m cp -r gs://qwiklabs-gcp-00-c908ca7e2977/fpc-manual.pdf .

mkdir: cannot create directory ‘test’: File exists
Copying gs://qwiklabs-gcp-00-c908ca7e2977/fpc-manual.pdf...
/ [1/1 files][  8.5 MiB/  8.5 MiB] 100% Done                                    
Operation completed over 1 objects/8.5 MiB.                                      


In [69]:
import os
import textract
from PyPDF2 import PdfReader

final_data = []

path = f"fpc-manual.pdf"

# loading pdf files, with page numbers as metadata.
reader = PdfReader(path)
for i, page in enumerate(reader.pages):

    text = page.extract_text()
    if text:
        packet = create_data_packet(
              file_name, file_type, page_number=int(i + 1), file_content=text
        )

        final_data.append(packet)


0 {'/BleedBox': [0, 0, 612, 792], '/Contents': [IndirectObject(1675, 0, 137129270400800), IndirectObject(1676, 0, 137129270400800), IndirectObject(1677, 0, 137129270400800), IndirectObject(1678, 0, 137129270400800), IndirectObject(1679, 0, 137129270400800), IndirectObject(1680, 0, 137129270400800), IndirectObject(1683, 0, 137129270400800), IndirectObject(1684, 0, 137129270400800)], '/CropBox': [0, 0, 612, 792], '/MediaBox': [0, 0, 612, 792], '/Parent': IndirectObject(1650, 0, 137129270400800), '/Resources': IndirectObject(1704, 0, 137129270400800), '/Rotate': 0, '/Thumb': IndirectObject(1087, 0, 137129270400800), '/TrimBox': [0, 0, 612, 792], '/Type': '/Page', '/u2pMat': [1, 0, 0, -1, 0, 792], '/xb1': 0, '/xb2': 612, '/xt1': 0, '/xt2': 612, '/yb1': 0, '/yb2': 792, '/yt1': 0, '/yt2': 792}
1 {'/BleedBox': [0, 0, 612, 792], '/Contents': IndirectObject(2, 0, 137129270400800), '/CropBox': [0, 0, 612, 792], '/MediaBox': [0, 0, 612, 792], '/Parent': IndirectObject(1650, 0, 137129270400800), '

In [73]:
# converting the data that has been read from GCS to Pandas DataFrame for easy readibility and downstream logic
import pandas as pd
pdf_data = pd.DataFrame.from_dict(final_data)
pdf_data.reset_index(inplace=True, drop=True)
pdf_data.head()

,page_number,content
0,1,The Health Code\nThese are regulations that we...
1,2,The United States has one of the\nsafest food ...
2,3,What is Ready-To- Eat Food?\nAny food product ...
3,4,Ice-Point Method\n/H20321/H20321Fill a contain...
4,5,Fresh fish\nThere is no inspection for fresh\n...


In [75]:
# init the vertexai package
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [96]:
# Load the text embeddings model
from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko@002")

In [97]:
import time
import tqdm  # to show a progress bar

# get embeddings for a list of texts
BATCH_SIZE = 5


def get_embeddings_wrapper(texts):
    embs = []
    for i in tqdm.tqdm(range(0, len(texts), BATCH_SIZE)):
        time.sleep(1)  # to avoid the quota error
        result = model.get_embeddings(texts[i : i + BATCH_SIZE])
        embs = embs + [e.values for e in result]
    return embs


In [98]:
# get embeddings for the question titles and add them as "embedding" column
pdf_data = pdf_data.assign(embedding=get_embeddings_wrapper(list(pdf_data.content)))
pdf_data.head()

100%|██████████| 19/19 [00:23<00:00,  1.25s/it]


,page_number,content,embedding
0,1,The Health Code\nThese are regulations that we...,"[-0.013569382950663567, -0.033456653356552124,..."
1,2,The United States has one of the\nsafest food ...,"[-0.01585172675549984, -0.012489386834204197, ..."
2,3,What is Ready-To- Eat Food?\nAny food product ...,"[-0.011286059394478798, -0.011021223850548267,..."
3,4,Ice-Point Method\n/H20321/H20321Fill a contain...,"[-0.013480892404913902, -0.017172927036881447,..."
4,5,Fresh fish\nThere is no inspection for fresh\n...,"[0.0021019037812948227, -0.018236981704831123,..."


In [101]:
import pandas as pd
import firebase_admin
from firebase_admin import firestore

# Initialize the Firestore client
#firebase_admin.initialize_app()
db = firestore.client()

# Get the PDF data
#pdf_data = pd.read_csv("pdf_data.csv", index_col=0)

# Iterate over the rows in the PDF data
for index, row in pdf_data.iterrows():
    # Get the page number
    page_number = row["page_number"]

    # Get the content and embedding
    content = row["content"]
    embedding = row["embedding"]

    # Create a document reference
    doc_ref = db.collection("pages").document(str(page_number))

    # Set the document data
    doc_ref.set({
        "content": content,
        "embedding": embedding
    })


In [102]:
import pandas as pd
import json

# Get the PDF data
#pdf_data = pd.read_csv("pdf_data.csv", index_col=0)

# Create a list of dictionaries, where each dictionary contains the page number and embedding
embeddings = []
for index, row in pdf_data.iterrows():
    embeddings.append({
        "page_number": row["page_number"],
        "embedding": row["embedding"]
    })

# Convert the list of dictionaries to JSON-L format
json_l = "\n".join(json.dumps(embedding) for embedding in embeddings)

# Write the JSON-L data to a file
with open("embeddings.json", "w") as f:
    f.write(json_l)


In [103]:
from google.cloud import storage

# Create a storage client
storage_client = storage.Client()

# Get the bucket
bucket = storage_client.bucket("qwiklabs-gcp-00-c908ca7e2977")

# Upload the file
bucket.blob("embeddings.json").upload_from_filename("embeddings.json")

# Print a success message
print("File uploaded successfully to Google Cloud Storage.")


File uploaded successfully to Google Cloud Storage.


In [124]:
# init the aiplatform package
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

BUCKET_URI = "gs://1qwiklabs-gcp-00-c908ca7e2977/embeddings.jsonl"

In [125]:
# create index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"assessment-index-endpoint-{UID}",
    contents_delta_uri=BUCKET_URI,
    dimensions=768,
    approximate_neighbors_count=20,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Creating MatchingEngineIndex
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Create MatchingEngineIndex backing LRO: projects/619606649119/locations/us-central1/indexes/2593884269365428224/operations/7604233996930646016
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex created. Resource name: projects/619606649119/locations/us-central1/indexes/2593884269365428224
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:To use this MatchingEngineIndex in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:index = aiplatform.MatchingEngineIndex('projects/619606649119/locations/us-central1/indexes/2593884269365428224')


In [126]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"assessment-index-endpoint-{UID}",
    public_endpoint_enabled=True,
)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Creating MatchingEngineIndexEndpoint
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Create MatchingEngineIndexEndpoint backing LRO: projects/619606649119/locations/us-central1/indexEndpoints/4094708845186646016/operations/2952015581856923648
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint created. Resource name: projects/619606649119/locations/us-central1/indexEndpoints/4094708845186646016
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:To use this MatchingEngineIndexEndpoint in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/619606649119/locations/us-central1/indexEndpoints/4094708845186646016')


In [127]:
DEPLOYED_INDEX_ID = f"embvs_tutorial_deployed_{UID}"

In [ ]:
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/619606649119/locations/us-central1/indexEndpoints/4094708845186646016
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/619606649119/locations/us-central1/indexEndpoints/4094708845186646016/operations/9112939872099762176


In [ ]:
test_embeddings = get_embeddings_wrapper(["what are potentially dangerous foods for children?"])

In [ ]:
# Test query
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=20,
)

# show the result
import numpy as np

for idx, neighbor in enumerate(response[0]):
    id = np.int64(neighbor.id)
    similar = df.query("id == @id", engine="python")
    print(f"{neighbor.distance:.4f} {similar.title.values[0]}")